# STAC Catalog 

Invoke a Common Workflow Language CommandLineTool to generate the STAC Catalog

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/cwl-cli/stac



## Setup

In [ ]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

curl -q -L https://github.com/eoap/zarr-cloud-native-format/releases/download/0.5.0/app-water-bodies.0.5.0.cwl > ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl 2> /dev/null

## Run the STAC Catalog generation CommandLineTool

Inspect and use `cwltool` to run the STAC Catalog generation definition:


In [ ]:
cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][6]' -

Run the CWL description, but first prepare the parameters.

The previous step generated the water bodies detection geotif:

In [ ]:
cat water-bodies-results.json 

Let's build the job parameters file with the otsu.tif file and the associated STAC Item input

In [ ]:
ostu_tif=$(cat water-bodies-results.json  | jq '.detected_water_body.path')

echo ${ostu_tif}

In [ ]:
item=$( cat convert-search-results.json | jq '.items[0]' )

echo ${item}

In [ ]:
cat <<EOF > stac-generation-params.yaml
item: 
- ${item}
rasters:
- class: File
  path: ${ostu_tif}
EOF

cat stac-generation-params.yaml | yq .

In [ ]:


cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl#stac \
    stac-generation-params.yaml > stac-generation-results.json 2> stac-generation.log

Let's look at the content of the stderr:

In [ ]:
cat stac-generation.log | egrep -v "WARNING|JSHINT"

Let's inspect the stdout produced. There are two `output` blocks with the id `aoi` and `items`. 

These are what the water bodies detection sub-workflow expects as inputs

In [ ]:
cat stac-generation-results.json | jq . -